In [1]:
import random
import numpy as np

job_times = [2, 4, 6, 8, 3, 5]
machines = 3
population_size = 10
generations = 100

def initialize_population(jobs, machines, pop_size):
    return [np.array([(j, random.randint(0, machines - 1)) for j in range(len(jobs))]) for _ in range(pop_size)]

def fitness(chromosome):
    loads = [0] * machines
    for job_id, machine_id in chromosome:
        loads[machine_id] += job_times[job_id]
    return 1 / (sum([l**2 for l in loads]) + 0.01)

def crossover(parent1, parent2):
    cut1, cut2 = sorted(random.sample(range(len(parent1)), 2))
    child1 = np.concatenate((parent1[:cut1], parent2[cut1:cut2], parent1[cut2:]))
    child2 = np.concatenate((parent2[:cut1], parent1[cut1:cut2], parent2[cut2:]))
    return child1, child2

def mutate(chromosome):
    idx = random.randint(0, len(chromosome) - 1)
    chromosome[idx] = (chromosome[idx][0], random.randint(0, machines - 1))
    return chromosome

def roulette_selection(population, fitnesses):
    total = sum(fitnesses)
    pick = random.uniform(0, total)
    current = 0
    for chrom, fit in zip(population, fitnesses):
        current += fit
        if current > pick:
            return chrom

population = initialize_population(job_times, machines, population_size)

for gen in range(generations):
    fitnesses = [fitness(chrom) for chrom in population]
    new_population = []
    for _ in range(population_size // 2):
        parent1 = roulette_selection(population, fitnesses)
        parent2 = roulette_selection(population, fitnesses)
        child1, child2 = crossover(parent1, parent2)
        if random.random() < 0.1:
            child1 = mutate(child1)
        if random.random() < 0.1:
            child2 = mutate(child2)
        new_population.extend([child1, child2])
    population = new_population

best_solution = max(population, key=fitness)
machine_loads = [0] * machines
for job_id, machine_id in best_solution:
    machine_loads[machine_id] += job_times[job_id]

print("Best job assignment:", best_solution)
print("Machine loads:", machine_loads)

Best job assignment: [[0 0]
 [1 2]
 [2 2]
 [3 0]
 [4 1]
 [5 1]]
Machine loads: [10, 8, 10]
